In [ ]:
# Code to reproduce figure 2c in Forlin, Mebrahtu et al

In [6]:
import warnings
warnings.filterwarnings("ignore")
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc



In [9]:
data_path = #insert data path here
adata = sc.read(data_path)
sc.pp.normalize_total(adata, target_sum = 1e3)


In [10]:
df_cytos = pd.read_csv('./data/cytokine_genelist.csv', index_col = [0])
hallmark_ifna = ast.literal_eval(df_cytos.loc[7, 'genes'])
hallmark_ifnb = ast.literal_eval(df_cytos.loc[8, 'genes'])



In [12]:
hallmark_list = [hallmark_ifna, hallmark_ifnb]

for i, hallmark in enumerate(['hallmark_ifna', 'hallmark_ifnb']):
    genes = hallmark_list[i]
    df_bootstrapped = pd.DataFrame(columns = ['mean_value', 'CellType', 'treatment'])
    for st in list(set(adata.obs.stim_treat_time)):
        temp = adata[adata.obs.stim_treat_time == st]
        for ct in ['B', 'CD4T', 'CD8T', 'Monocyte', 'DC', 'Eosinophil', 'NK', 'Neutrophil', 'pDC', 'Plasmablast']:
            temp2 = temp[temp.obs.CellType == ct]
            temp2 = temp2[:, temp2.var.index.isin(genes)].to_df()
            
            summ = np.sum(temp2, axis = 1)
            df_ctrl = adata[(adata.obs.stim_treat_time == 'IFNb_Ctrl_2h')&(adata.obs.CellType == ct), adata.var.index.isin(genes)].to_df()
            ctrl_summ = np.sum(df_ctrl, axis = 1)
            
            for i in range(100):
                mean_value = np.mean(summ.sample(len(summ), replace = True))
                df_bootstrapped.loc[st + '_' + ct + '_' + str(i)] = [mean_value, ct, st]
    df_bootstrapped = df_bootstrapped.reset_index().drop('index', axis = 1)
    df_bootstrapped['hallmark'] = hallmark
    #df_bootstrapped.to_csv(f'./data/Figure2_{hallmark}_toR.csv', sep = ',')
            

In [5]:
df_bootstrapped

,mean_value,CellType,treatment,hallmark
0,8.827572,B,IFNb_Ctrl_2h,hallmark_ifnb
1,9.028873,B,IFNb_Ctrl_2h,hallmark_ifnb
2,8.629498,B,IFNb_Ctrl_2h,hallmark_ifnb
3,8.778787,B,IFNb_Ctrl_2h,hallmark_ifnb
4,8.741452,B,IFNb_Ctrl_2h,hallmark_ifnb
...,...,...,...,...
6995,4.634559,Plasmablast,IFNb_BAR1000_2h,hallmark_ifnb
6996,6.160433,Plasmablast,IFNb_BAR1000_2h,hallmark_ifnb
6997,7.086497,Plasmablast,IFNb_BAR1000_2h,hallmark_ifnb
6998,5.093172,Plasmablast,IFNb_BAR1000_2h,hallmark_ifnb
